DECISION TREE (RMSE= 17541)


In [9]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Carica i dataset
train_data = pd.read_csv('/Users/gersiprendushi/Downloads/save_train.csv')  # Cambia con il tuo file di training
test_data = pd.read_csv('/Users/gersiprendushi/Downloads/save_test.csv')    # Cambia con il tuo file di test
submission_example = pd.read_csv('/Users/gersiprendushi/Desktop/home-data-for-ml-course/sample_submission.csv')  # File con gli ID

# Separazione delle features e della variabile target nel dataset di training
X_train = train_data.drop(columns=['SalePrice'])  # Sostituisci 'SalePrice' con il nome della tua colonna target
y_train = train_data['SalePrice']  # Sostituisci con il nome della tua colonna target

# Allena il modello (RandomForest come esempio)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Previsione sul dataset di test (non contiene la colonna target)
X_test = test_data  # Non c'è bisogno di rimuovere 'SalePrice' dal test, poiché non è presente
predictions = model.predict(X_test)

# Estrai gli ID dal file di esempio della submission
submission_ids = submission_example.iloc[:, 0]  # Prende la prima colonna (ID)

# Crea il DataFrame per la submission
results = pd.DataFrame({
    submission_example.columns[0]: submission_ids,  # Mantiene il nome originale della colonna ID
    submission_example.columns[1]: predictions  # Mantiene il nome originale della colonna delle previsioni
})

# Salva il DataFrame come file CSV
results.to_csv('/Users/gersiprendushi/Desktop/submission_dectree.csv', index=False)

# Calcolare l'RMSE sul training set come verifica
y_train_pred = model.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(f"Training RMSE: {rmse}")

Training RMSE: 11492.294087297303


LASSO REGRESSION (RMSE=19146)


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA
import time

# Carica i dataset
train_data = pd.read_csv('/Users/gersiprendushi/Downloads/save_train.csv')
test_data = pd.read_csv('/Users/gersiprendushi/Downloads/save_test.csv')
submission_example = pd.read_csv('/Users/gersiprendushi/Desktop/home-data-for-ml-course/sample_submission.csv')  # File con ID

# Rimuovi ID dalle colonne (se presente)
train_data.drop(columns=['Id'], errors='ignore', inplace=True)
test_data.drop(columns=['Id'], errors='ignore', inplace=True)

# Separa le feature dal target
X = train_data.drop(columns=['SalePrice'])
y = train_data['SalePrice']
X_test = test_data.copy()

# Normalizza SOLO le feature
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)  
X_test_scaled = scaler.transform(X_test)  

# Aggiungi PCA per ridurre la dimensionalità
pca = PCA(n_components=0.95)  # Mantieni il 95% della varianza
X_pca = pca.fit_transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Riduci il numero di valori di alpha per GridSearchCV
alphas = np.logspace(-2, 2, 5)  # Fai meno prove

# Esegui GridSearchCV per trovare il miglior alpha
grid = GridSearchCV(Lasso(max_iter=10000, tol=1e-4), {'alpha': alphas}, cv=3, scoring='neg_root_mean_squared_error')
start_time = time.time()
grid.fit(X_pca, y)
print(f"GridSearchCV completato in {time.time() - start_time:.2f} secondi")

best_alpha = grid.best_params_['alpha']
print(f"Miglior valore di alpha trovato: {best_alpha}")

# Allena il modello Lasso con il miglior alpha
lasso_model = Lasso(alpha=best_alpha, max_iter=10000, tol=1e-3)
lasso_model.fit(X_pca, y)
lasso_predictions = lasso_model.predict(X_test_pca)

# Prepara il DataFrame per la submission
submission_lasso = pd.DataFrame({
    'Id': submission_example['Id'],
    'SalePrice': lasso_predictions
})

# Salva il file CSV per la competition
submission_lasso.to_csv('/Users/gersiprendushi/Desktop/submission_lasso.csv', index=False)
print("✅ File CSV per Lasso salvato con successo!")


GridSearchCV completato in 0.10 secondi
Miglior valore di alpha trovato: 100.0
✅ File CSV per Lasso salvato con successo!


LINEAR REGRESSION (RMSE= 18483)

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
import time

# Carica i dataset
train_data = pd.read_csv('/Users/gersiprendushi/Downloads/save_train.csv')
test_data = pd.read_csv('/Users/gersiprendushi/Downloads/save_test.csv')
submission_example = pd.read_csv('/Users/gersiprendushi/Desktop/home-data-for-ml-course/sample_submission.csv')  # File con ID

# Rimuovi ID dalle colonne (se presente)
train_data.drop(columns=['Id'], errors='ignore', inplace=True)
test_data.drop(columns=['Id'], errors='ignore', inplace=True)

# Separa le feature dal target
X = train_data.drop(columns=['SalePrice'])
y = train_data['SalePrice']
X_test = test_data.copy()

# Inizializza il modello di regressione lineare
lr_model = LinearRegression()

# Esegui GridSearchCV (anche se il modello di regressione lineare di base non ha parametri da ottimizzare)
param_grid = {'fit_intercept': [True, False]}  # Puoi esplorare anche altre opzioni, ma il fit_intercept è il parametro principale
grid = GridSearchCV(lr_model, param_grid, cv=3, scoring='neg_root_mean_squared_error')

# Avvia il grid search
start_time = time.time()
grid.fit(X, y)

# Mostra il miglior parametro e tempo di esecuzione
best_fit_intercept = grid.best_params_['fit_intercept']
print(f"Miglior parametro 'fit_intercept': {best_fit_intercept}")
print(f"Tempo di esecuzione: {time.time() - start_time:.2f} secondi")

# Allena il modello con il miglior parametro
lr_model = LinearRegression(fit_intercept=best_fit_intercept)
lr_model.fit(X, y)

# Fai le previsioni sul dataset di test
lr_predictions = lr_model.predict(X_test)

# Prepara il DataFrame per la submission
submission_lr = pd.DataFrame({
    'Id': submission_example['Id'],  # Usa gli ID originali
    'SalePrice': lr_predictions
})

# Salva il file CSV per la competition
submission_lr.to_csv('/Users/gersiprendushi/Desktop/submission_lr.csv', index=False)

print("✅ File CSV per Regressione Lineare salvato con successo!")

Miglior parametro 'fit_intercept': True
Tempo di esecuzione: 0.86 secondi
✅ File CSV per Regressione Lineare salvato con successo!


LIGHTGBM (RMSE=15593)

In [40]:
import pandas as pd
import lightgbm as lgb
import numpy as np
# ---------------------
# STEP 1: Caricamento e Preprocessing
# ---------------------

# Caricamento del dataset di training già pulito
train_data = pd.read_csv('/Users/gersiprendushi/Downloads/save_train.csv')

# Separazione delle feature (X) e del target (y)
X_train = train_data.drop(columns=['SalePrice'])
y_train = train_data['SalePrice']

# Converte in formato LightGBM (dtrain)
dtrain = lgb.Dataset(data=X_train, label=y_train)

# ---------------------
# STEP 2: Addestramento del modello LightGBM
# ---------------------

params = {
    'objective': 'regression',          # Obiettivo di regressione
    'metric': 'rmse',                   # Errore quadratico medio
    'max_depth': 6,                     # Profondità massima degli alberi
    'learning_rate': 0.05,              # Tasso di apprendimento
    'num_leaves': 31,                   # Numero di foglie per albero
    'subsample': 0.8,                   # Campioni per albero
    'colsample_bytree': 0.8,            # Feature per albero
    'min_data_in_leaf': 20              # Dati minimi per foglia
}

# Addestramento del modello (senza cross-validation e senza verbose)
lgb_model = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=500,               # Numero massimo di iterazioni
)

# ---------------------
# STEP 3: Previsioni sul training set
# ---------------------

# Previsioni sul training set
y_pred_train = lgb_model.predict(X_train)

# Calcolo delle metriche di performance
rmse = np.sqrt(np.mean((y_pred_train - y_train)**2))
rsquared = np.corrcoef(y_pred_train, y_train)[0, 1]**2

# Stampa delle metriche
print(f"RMSE: {rmse:.2f}")
print(f"R²: {rsquared:.2f}")

# Valutazione della qualità del modello
if rmse < 50000 and rsquared > 0.7:
    print("✅ Il modello ha buone prestazioni!")
else:
    print("⚠️ Il modello potrebbe non essere ottimale. Considera di migliorarlo.")

# ---------------------
# STEP 4: Preprocessing del Test Set
# ---------------------

# Caricamento del dataset di test già pulito
test_data = pd.read_csv('/Users/gersiprendushi/Downloads/save_test.csv')

# Mantiene solo le colonne presenti nel training set
X_test = test_data[X_train.columns]

# ---------------------
# STEP 5: Previsioni sul Test Set
# ---------------------

lgb_predictions = lgb_model.predict(X_test)

# ---------------------
# STEP 6: Creazione del file di Submission
# ---------------------

# Caricamento del file di esempio per la submission
submission_example = pd.read_csv('/Users/gersiprendushi/Desktop/home-data-for-ml-course/sample_submission.csv')

# Sostituzione dei valori di SalePrice con le previsioni 
submission_example['SalePrice'] = lgb_predictions

# Creazione e salvataggio del file di submission
submission_example.to_csv('/Users/gersiprendushi/Desktop/lgb_submission.csv', index=False)

# Messaggio di conferma
print("File di submission creato con successo: lgb_submission.csv")

RMSE: 10448.73
R²: 0.98
✅ Il modello ha buone prestazioni!
File di submission creato con successo: lgb_submission.csv


XGBOOST (RMSE= 15668)

In [33]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.stats import uniform, randint

# STEP 1: Preprocessing
# Caricamento del dataset di training già pulito
train_data = pd.read_csv('/Users/gersiprendushi/Downloads/save_train.csv')

# Separazione delle feature (X) e del target (y)
X_train = train_data.drop(columns=['SalePrice'])
y_train = train_data['SalePrice']

# Conversione in matrice DMatrix per XGBoost
dtrain = xgb.DMatrix(data=X_train, label=y_train)

# STEP 2: Ricerca su griglia dei migliori parametri (aggiustata per un'ottimizzazione migliore)
param_grid = {
    'max_depth': randint(3, 10),  # Profondità degli alberi
    'eta': uniform(0.01, 0.3),  # Learning rate
    'subsample': uniform(0.7, 0.3),  # Percentuale di dati usati
    'colsample_bytree': uniform(0.7, 0.3),  # Percentuale di feature usate
    'lambda': uniform(0, 10),  # Regolarizzazione L2
    'alpha': uniform(0, 10),  # Regolarizzazione L1
    'objective': ['reg:squarederror'],  # Tipo di obiettivo
    'eval_metric': ['rmse']
}

# Definire il modello XGBoost
xgb_model = xgb.XGBRegressor()

# RandomizedSearchCV per la ricerca dei parametri
random_search = RandomizedSearchCV(
    estimator=xgb_model, 
    param_distributions=param_grid,
    n_iter=50,  # Numero di combinazioni da testare
    scoring='neg_root_mean_squared_error',  # RMSE come metrica
    n_jobs=-1,  # Usa tutti i core disponibili
    cv=5,  # 5-fold Cross Validation
    verbose=2
)

# Esegui la ricerca sui parametri
random_search.fit(X_train, y_train)

# Best parameters
best_params = random_search.best_params_
print("Migliori parametri trovati:")
print(best_params)

# STEP 3: Addestramento finale con i migliori parametri
best_model = random_search.best_estimator_

# STEP 4: Risultati del train
# Previsioni sul training set
y_pred_train = best_model.predict(X_train)

# Calcolo dell'errore quadratico medio (RMSE)
rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))

# Calcolo del coefficiente di determinazione (R²)
rsquared = r2_score(y_train, y_pred_train)

# Stampa delle metriche
print(f"RMSE: {round(rmse, 2)}")
print(f"R²: {round(rsquared, 15)}")

# STEP 5: Preprocessing del test set
# Caricamento del dataset di test già pulito
test_data = pd.read_csv('/Users/gersiprendushi/Downloads/save_test.csv')

# Previsioni con il modello finale
predictions = best_model.predict(test_data)

# STEP 6: Creazione del file di submission
# Caricamento del file di esempio per la submission
sample_submission = pd.read_csv('/Users/gersiprendushi/Desktop/home-data-for-ml-course/sample_submission.csv')

# Sostituzione dei valori di SalePrice con le previsioni 
sample_submission['SalePrice'] = predictions

# Salvataggio del file di submission
submission_path = '/Users/gersiprendushi/Desktop/sample_submission.csv'
sample_submission.to_csv(submission_path, index=False)

# Messaggio di conferma
print(f"File di submission creato con successo: {submission_path}")

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Migliori parametri trovati:
{'alpha': 2.2082428411043153, 'colsample_bytree': 0.7460747537383681, 'eta': 0.2379619722811321, 'eval_metric': 'rmse', 'lambda': 7.927183641101987, 'max_depth': 4, 'objective': 'reg:squarederror', 'subsample': 0.9982699249662286}
RMSE: 10754.28
R²: 0.981661893386516
File di submission creato con successo: /Users/gersiprendushi/Desktop/sample_submission.csv


RANDOM FOREST (RMSE=

SVM (RMSE = 22000..)

In [43]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# ---------------------
# STEP 1: Caricamento e Preprocessing
# ---------------------

# Caricamento del dataset di training già pulito
train_data = pd.read_csv('/Users/gersiprendushi/Downloads/save_train.csv')

# Separazione delle feature (X) e del target (y)
X_train = train_data.drop(columns=['SalePrice'])
y_train = train_data['SalePrice']

# Trasformazione logaritmica del target
y_train = np.log1p(y_train)

# Gestione dei valori mancanti (nel caso in cui ci siano)
X_train = X_train.fillna(X_train.mean())

# Scaling delle feature numeriche
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

# Creazione di un set di validazione
X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

# ---------------------
# STEP 2: Addestramento del modello SVM
# ---------------------

# Inizializzazione del modello SVM con kernel RBF
svm_model = SVR(kernel='rbf')

# Addestramento del modello SVM
svm_model.fit(X_train, y_train)

# ---------------------
# STEP 3: Risultati del training
# ---------------------

# Previsioni sul training set
y_pred_train = svm_model.predict(X_train)

# Calcolo dell'errore quadratico medio (RMSE)
rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))

# Calcolo del coefficiente di determinazione (R²)
rsquared = r2_score(y_train, y_pred_train)

# Stampa delle metriche
print(f"RMSE: {rmse:.2f}")
print(f"R²: {rsquared:.2f}")

# Valutazione delle performance del modello
rmse_threshold = 50000  # Soglia indicativa per un buon RMSE
rsquared_threshold = 0.7  # Un buon modello ha R² >= 0.7

if rmse < rmse_threshold and rsquared > rsquared_threshold:
    print("Il modello ha buone prestazioni!")
else:
    print("Il modello potrebbe non essere ottimale, considera di migliorarlo.")

# ---------------------
# STEP 4: Preprocessing del test set
# ---------------------

# Caricamento del dataset di test già pulito
test_data = pd.read_csv('/Users/gersiprendushi/Downloads/save_test.csv')

# Applicazione delle stesse trasformazioni al test set
X_test = test_data[X_train.columns]
X_test = X_test.fillna(X_test.mean())
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

# ---------------------
# STEP 5: Previsioni sul Test Set
# ---------------------

# Previsioni con il modello SVM
predictions = svm_model.predict(X_test_scaled)

# ---------------------
# STEP 6: Creazione del file di submission
# ---------------------

# Caricamento del file di esempio per la submission
submission_example = pd.read_csv('/Users/gersiprendushi/Desktop/home-data-for-ml-course/sample_submission.csv')

# Sostituzione dei valori di SalePrice con le previsioni
submission_example['SalePrice'] = np.expm1(predictions)  # Inverso della logaritmica

# Creazione e salvataggio del file di submission
submission_example.to_csv('/Users/gersiprendushi/Desktop/home-data-for-ml-course/svm_submission_2.csv', index=False)

# Messaggio di conferma
print("File di submission creato con successo: svm_submission_2.csv")

RMSE: 0.08
R²: 0.96
Il modello ha buone prestazioni!
File di submission creato con successo: svm_submission_2.csv
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=None, max_features=aut